# Common - Package import

In [5]:
# print_function for compatibility with Python 3
from __future__ import print_function
print('print function is ready to serve')

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline

print function is ready to serve


In [5]:
# NumPy for numerical computing
import numpy as np
np.random.seed(923)
import random
random.seed(923)

# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_column', 100)

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam

import tensorflow as tf

import os
import gc
import threading
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

Using TensorFlow backend.


In [7]:
import sys
sys.path.append('./utils')

from data import Data
from models import Models, TargetStopping
from tags import Tags
tags = Tags()

In [8]:
PLANET_KAGGLE_ROOT = '/data/planet-data/'
if not os.path.exists(PLANET_KAGGLE_ROOT):
    PLANET_KAGGLE_ROOT = '/Users/jiayou/Documents/Kaggle Data/Amazon'

N_TAGS = 17
N_TRAIN = 40479
N_TEST_T = 40669
N_TEST_F = 20522
N_TEST = N_TEST_T + N_TEST_F

# Training

In [9]:
def train_on_gpu(val=0, toy=None, d=None, name='', gpu=0):
    g = tf.Graph()
    with g.as_default():
        sess = tf.Session()
        with sess.as_default():
            with tf.device('/gpu:{}'.format(gpu)):
                train(val=val, toy=toy, d=d, name=name)
    
def train(val=0, toy=None, d=None, name=''):
    print('')
    print('Training {} with val = {}'.format(name, val))
    print('')
    
    if d is None:
        d = Data(tif=False, toy=toy)
        
    gen_val = None
    val_steps = None
    if val is not None:
        gen_val = d.gen_val(100, val=val)
        val_steps = 80

    m = Models.new_incnet()
    
    for layer in m.layers:
        layer.trainable = False
    m.layers[-1].trainable = True
    m.compile(metrics=[Models.amazon_score, 'accuracy'],
              loss='binary_crossentropy',
              optimizer=Adam(lr=0.001))
    h = m.fit_generator(
        d.gen_train(32, val=val), steps_per_epoch=1300,
        epochs=1, initial_epoch=0,
        validation_data=gen_val, validation_steps=val_steps,
        max_q_size=10)    
    
    for layer in m.layers:
        layer.trainable = True
    lrs = [1e-4, 1e-5, 1e-6]
    epochs = [6, 3, 3]
    loss_stop = np.random.uniform(0.059, 0.065)
    initial_epoch = 1
    for lr, epoch in zip(lrs, epochs):
        m.compile(metrics=[Models.amazon_score, 'accuracy'],
                  loss='binary_crossentropy',
                  optimizer=Adam(lr=lr))
        h = m.fit_generator(
            d.gen_train(32, val=val), steps_per_epoch=1300,
            epochs=initial_epoch + epoch, initial_epoch=initial_epoch,
            validation_data=gen_val, validation_steps=val_steps,
            callbacks=[
                ModelCheckpoint('weights-{}.hdf5'.format(name), monitor='loss', save_best_only=True, verbose=1),
                TargetStopping(monitor='loss', target=loss_stop, verbose=1)
            ],
            max_q_size=10, verbose=0)
        initial_epoch += epoch
        
    return h

def tune_on_gpu(val=0, d=None, name='', gpu=0):
    g = tf.Graph()
    with g.as_default():
        sess = tf.Session()
        with sess.as_default():
            with tf.device('/gpu:{}'.format(gpu)):
                tune(val=val, d=d, name=name)

def tune(name='', val=0, d=None):
    print('')
    print('Tuning {} with val = {}'.format(name, val))
    print('')
    
    if d is None:
        d = Data(tif=False)
        
    gen_val = None
    val_steps = None
    if val is not None:
        gen_val = d.gen_val(100, val=val)
        val_steps = 80

    m = Models.load_incnet('weights-{}.hdf5'.format(name))
    m.compile(metrics=['accuracy'],
              loss='binary_crossentropy',
              optimizer=Adam(lr=1e-6))

    h = m.fit_generator(
        d.gen_train(32, val=val), steps_per_epoch=1300,
        epochs=15, initial_epoch=13,
        validation_data=gen_val, validation_steps=val_steps,
        callbacks=[
            ModelCheckpoint('weights-{}-tune.hdf5'.format(name), monitor='loss', save_best_only=True, verbose=1)
        ],
        max_q_size=10, verbose=0)
        
    return h

In [ ]:
# Download pretrained weights to local disk
Models.new_incnet()

In [ ]:
# Parallel ensemble training
toy = None
d = Data(tif=False, toy=toy)
ts = []
for i in range(8):
    t = threading.Thread(
        target=train_on_gpu, 
        kwargs={'val': None, 'd': d, 'toy': None, 'name': 'v13-n{}'.format(i), 'gpu': i})
    t.start()
    ts.append(t)
for i in range(len(ts)):
    ts[i].join()

In [1]:
# Ensemble training
toy = 10000
results = []
d = Data(tif=False, toy=toy)
for i in [0]:
    r = train(val=i, d=d, toy=toy)
    results.append(r)
    gc.collect()

In [ ]:
# Parallel tuning
toy = None
d = Data(tif=False, toy=toy)
ts = []
for i in range(8):
    t = threading.Thread(
        target=tune_on_gpu, 
        kwargs={'val': None, 'd': d, 'name': 'v13-n{}'.format(i), 'gpu': i})
    t.start()
    ts.append(t)
for i in range(len(ts)):
    ts[i].join()

# Tag fine tuning

In [1]:
def mask_weights(w, select):
    for i in range(N_TAGS):
        if not i in select:
            w[318][:,i] = 0
            w[319][i] = -1e7
    return w

def tune_tag(weights, select=[], val=0, toy=None, d=None):
    print('')
    print('Training with val = {}'.format(val))
    print('')
    
    if d is None:
        d = Data(tif=False, toy=toy)

    m = Models.load_resnet50(weights)
    m.set_weights(mask_weights(m.get_weights(), select))
    m.compile(metrics=['accuracy'],
              loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001))

    h = m.fit_generator(
        d.gen_mask(d.gen_train(32, val=val), select=select), steps_per_epoch=1000,
        epochs=10, initial_epoch=0,
        validation_data=d.gen_mask(d.gen_val(100, val=val), select=select), validation_steps=80,
        callbacks=[
            ModelCheckpoint('weights-v9-f{}-tune.hdf5'.format(val), save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=0, min_lr=5e-7, verbose=1)],
        max_q_size=10)
    
    return h

In [ ]:
pred = None
pred8 = None

def predict_val(toy=None, batch_size=20, weights='', d=None, val=0):
    model = Models.load_resnet50(weights)
    print('Model weights loaded')
    
    if d is None:
        d = Data(toy=toy)
    
    cnt = 0
    global pred
    global pred8
    n = len(d.y[val])
    pred = np.zeros((n, N_TAGS))
    pred8 = np.zeros((n * 8, N_TAGS))
    
    print('Start predicting..')
    for X in d.gen_val_augmented(batch_size, val=val):
        y = model.predict_on_batch(X)
        k = int(len(y) / 8 + 0.1)
        pred8[cnt*8:(cnt+k)*8,:] = y[:,:]
        for i in range(k):
            pred[cnt+i,:] = d.consolidate(y[8*i:8*(i+1),:])
        cnt += k
        print('Predicted {} images'.format(cnt))
    print('Predicted all {} images'.format(cnt))

In [19]:
select = [2, 4, 7, 12]
val = 4
d = Data(tif=False)

In [3]:
tune_tag('weights-v9-f{}.hdf5'.format(val), select=select, val=val, d=d)

In [4]:
predict_val(d=d, weights='weights-v9-f{}-tune.hdf5'.format(val), val=val)

In [5]:
tags.plot_roc(pred, d.y[val], title='Fine tune tags')

0.597374889411837